In [17]:
import zipfile

zip_path = "/content/Business-Quant-Dataset-Html-Tables.zip"
extract_path = "/content/Business_Quant_Extracted"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")


Extraction complete!


In [18]:
!pip install pandas beautifulsoup4 lxml


In [19]:
import pandas as pd
import os
from bs4 import BeautifulSoup

# Define the input folder and output CSV file
input_folder = "Business_Quant_Extracted/Business Quant Dataset - Html Tables"
output_csv = "extracted_table_data.csv"

data_list = []

# Iterate through all extracted HTML files
for file_name in os.listdir(input_folder):
    if file_name.endswith(".html"):
        file_path = os.path.join(input_folder, file_name)

        # Read HTML content
        with open(file_path, "r", encoding="utf-8") as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Extract the table title dynamically
        table_title = "-"
        possible_title_tags = ["h1", "h2", "h3", "h4", "h5", "h6", "caption", "title"]
        for tag in possible_title_tags:
            title_element = soup.find(tag)
            if title_element and title_element.text.strip():
                table_title = title_element.text.strip()
                break  # Stop at the first valid title found

        # Ensure the correct title for specific tables
        if "table9" in file_name.lower() or "table_9" in file_name.lower():
            table_title = "Three months ended March 31"

        # Extract tables dynamically
        tables = pd.read_html(file_path)

        for table in tables:
            # Identify the first column as label
            label_column = table.columns[0]

            # Reshape table into long format
            table_melted = table.melt(id_vars=[label_column], var_name="column header", value_name="value")

            # Rename first column to "label"
            table_melted.rename(columns={label_column: "label"}, inplace=True)

            # Add metadata columns
            table_melted.insert(0, "filename", file_name.replace(".html", ""))
            table_melted.insert(2, "tabletitle", table_title)

            # Append to data list
            data_list.append(table_melted)

# Combine all extracted tables into a single DataFrame
final_df = pd.concat(data_list, ignore_index=True)
final_df.to_csv(output_csv, index=False)

print(f"Extraction complete! Data saved to {output_csv}")


Extraction complete! Data saved to extracted_table_data.csv


In [20]:
from google.colab import files
files.download("extracted_table_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>